### Computation of the baseline

#### Load relevante moduler

In [1]:
include("../ToolBox/ToolBox.jl")
using .ToolBox
using .Geometry
using .SlcUtil
using .Load
using .Misc
using PyCall
scipy_interp = pyimport("scipy.interpolate");
ndimage = pyimport("scipy.ndimage");
rasterio = PyCall.pyimport("rasterio")

using LinearAlgebra
using Dates

proj_create: init=epsg:/init=IGNF: syntax not supported in non-PROJ4 emulation mode


In [2]:
"""
Load dem subset, wraps dem function.
"""
function dem_subset(dem_path, meta, view; nan_fill= 0, padding=[90,90])
    footprint = SlcUtil.footprint(meta, view)
    latlon_window = ((minimum(footprint[1]),maximum(footprint[1])),(minimum(footprint[2]),maximum(footprint[2])))
    dem = Load.dem(dem_path, latlon_window; nan_fill= 0, padding=padding);
    return dem
end

function mosaic2normal_view(mosaic_view, meta)
    @assert mosaic_view[1].start >= meta["burst_meta"]["first_line_mosaic"][1]
    @assert mosaic_view[1].stop <= (meta["burst_meta"]["first_line_mosaic"][end] + meta["lines_per_burst"])
    mosaic_view
    
    start_burst = findlast(
    meta["burst_meta"]["first_line_mosaic"].<= mosaic_view[1].start)

    lines_in_first_burst = mosaic_view[1].start -meta["burst_meta"]["first_line_mosaic"][start_burst] +1
    line_start = lines_in_first_burst + (start_burst-1)* meta["lines_per_burst"]

    end_burst = 0
    if mosaic_view[1].stop < meta["lines_per_burst"]
        end_burst = 1
    else
        end_burst = findlast(
            (meta["burst_meta"]["first_line_mosaic"] .+meta["lines_per_burst"] ).< mosaic_view[1].stop)+1
    end

    lines_in_last_burst =  mosaic_view[1].stop - meta["burst_meta"]["first_line_mosaic"][end_burst] +1
    line_end = lines_in_last_burst+ (end_burst-1)* meta["lines_per_burst"]
    return [line_start:line_end,mosaic_view[2]]
end
    
function extract_datetime(SAFE_path; start_date=true)
    extract_SAFE_name = split(SAFE_path, "/")[end]
    if start_date
        date_string = split(extract_SAFE_name, "_")[6]
    else
        date_string = split(extract_SAFE_name, "_")[7]
    end
    year = date_string[1:4]
    month = date_string[5:6]
    day = date_string[7:8]
    hour = date_string[10:11]
    minute = date_string[12:13]
    second = date_string[14:end]
    date_int = parse.(Int, [year, month, day, hour, minute, second])
    return DateTime(date_int...)
end

function days_between_acquisitions(date1, date2)
    return Dates.value(Date(date1) - Date(date2))
end

days_between_acquisitions (generic function with 1 method)

In [3]:
#input: view, dem, meta
function get_elevation(view, dem, meta, pod)
    # initilize lut
    line, sample = Misc.flatten(collect(view[1]), collect(view[2])) # line, sample

    # Get heights
    lat_dem, lon_dem, heights = Misc.flatten(dem...)
    line_sample_dem = to_line_sample(hcat(lat_dem, lon_dem), heights, pod..., meta);
    elevation = Misc.interp(line_sample_dem[:,1], line_sample_dem[:,2], heights,
                            line, sample)
    @assert sum(isnan.(elevation)) == 0
    return elevation
end

function satellite_orbit(line, meta, pod)
    azimuth_time = meta["t_start"] + (line - 1) * 1 / meta["azimuth_frequency"]
    state_vectors_poly, state_vectors_mean, state_vectors_std = Geometry.satellite_trajectory(pod[1], pod[2], meta["t_start"], meta["t_stop"]);
    tmp = Geometry.polyval_state_vectors(state_vectors_poly, azimuth_time, state_vectors_mean, state_vectors_std)
    sat2_position, sat2_velocity = tmp[1:3], tmp[4:6]
    
    return sat2_position, sat2_velocity
end

function line_of_sight_point(line, pixel_position, meta, pod)
    sat1_position, _ = satellite_orbit(line, meta, pod)
    los = sat1_position .- pixel_position
    return los
end

function baseline_perpendicular(mosaic_view, dem, meta, pod)
    # get sample and line corresponding to view
    line1, sample1 = Misc.flatten(mosaic_view[1], mosaic_view[2])  # line, sample. Needed input structure for to_lat_lon
    
    # get elevation at view
    height = get_elevation(mosaic_view, dem, meta[1], pod[1])
    
    # compute lat, lon from line sample and height
    lat, lon = Geometry.to_lat_lon(hcat(line1, sample1), height, pod[1][1], pod[1][2], meta[1])
    
    # get the position of the pixel on ground in cartesian coordinates
    pixel_position = Geometry.ellipsoid2xyz(deg2rad(lat), deg2rad(lon), height...)
    
    # compute the line of sight vector
    los1 = line_of_sight_point(line1[1], pixel_position, meta[1], pod[1])
    
    # compute the 2nd line of sight vector based on lat lon on ground
    line2, sample2 = to_line_sample(hcat(lat, lon), height, pod[2][1], pod[2][2], meta[2])
    los2 = line_of_sight_point(line2[1], pixel_position, meta[2], pod[2])
    
    # find the correct sign
    s = sign(norm(los1) - norm(los2));
    
    # the area of the parallelogram spanned by the unit vector and the direct baseline is equal to the perpendicular baseline
    perpendicular_baseline = s * norm(cross(los1 - los2, los1 ./ norm(los1)))
    return perpendicular_baseline
end

baseline_perpendicular (generic function with 1 method)

## Load relevant data

##### Paths

In [4]:
dem_path = "/Users/eyu/local_data/data/srtm_38_01/srtm_38_01.tif"
slc1_safe_path = "/Users/eyu/local_data/data/phase_bug/BB/S1B_IW_SLC__1SDV_20170408T053951_20170408T054019_005065_008DBC_AEEF.SAFE"
slc2_safe_path = "/Users/eyu/local_data/data/phase_bug/BB/S1B_IW_SLC__1SDV_20170420T053952_20170420T054019_005240_0092C6_3820.SAFE";

##### Load SLC1

In [5]:
slc1_data_path, slc1_meta_path, slc1_calibration_path = Load.slc_paths(slc1_safe_path, "VV", 3);
slc1_meta = Load.slc_meta(slc1_meta_path);
slc1_pod = Load.precise_orbit(Load.pod_path(slc1_meta["t_0"], slc1_meta["mission_id"],
                        "/Users/eyu/local_data/data/phase_bug/POD"), slc1_meta["t_0"]);
#slc1_start_time, slc1_stop_time = UnitTest.meta_start_datetime(slc1_meta_path);


##### Load SLC2

In [6]:
slc2_data_path, slc2_meta_path, slc2_calibration_path = Load.slc_paths(slc2_safe_path, "VV", 3);
slc2_meta = Load.slc_meta(slc2_meta_path);
slc2_pod = Load.precise_orbit(Load.pod_path(slc2_meta["t_0"], slc2_meta["mission_id"],
                        "/Users/eyu/local_data/data/phase_bug/POD"), slc2_meta["t_0"]);

##### Load DEM

In [7]:
padding = 0;
burst_middle = Int((slc1_meta["lines_per_burst"] * slc1_meta["burst_count"]) / 2);
swath_middle = Int((slc1_meta["samples_per_burst"]) / 2);
mosaic_view = [(burst_middle):(burst_middle), swath_middle:swath_middle];  # in mosaic geometry
#mosaic_view = [2104:2104, 12665:12665];  # in mosaic geometry

interest_view = mosaic2normal_view(mosaic_view, slc1_meta);  # in raw geometry

In [8]:
dem = dem_subset(dem_path, slc1_meta, interest_view; nan_fill= 0, padding=[180,180]);

proj_create: init=epsg:/init=IGNF: syntax not supported in non-PROJ4 emulation mode


## Compute perpendicular baseline

In [9]:
baseline_perpendicular(interest_view, dem, [slc1_meta, slc2_meta], [slc1_pod, slc2_pod])

┌ Warning: `polyfit(x, y, n=length(x) - 1, sym=:x)` is deprecated, use `fit(Poly, x, y, n; var=sym)` instead.
│   caller = polyfit(::Array{Float64,1}, ::Array{Float64,1}, ::Int64) at deprecated.jl:56
└ @ Polynomials ./deprecated.jl:56


-48.624954511700345

In [10]:
days_between_acquisitions(extract_datetime(slc1_safe_path), extract_datetime(slc2_safe_path))

-12

# Step by step solution:

#### Get line, sample, height from point of interest

In [11]:
line1, sample1 = Misc.flatten(interest_view[1], interest_view[2])  # line, sample. Needed input structure for to_lat_lon
height = get_elevation(interest_view, dem, slc1_meta, slc1_pod)

1-element Array{Float64,1}:
 0.0

#### Compute the corresponding lat lon coordinates, and convert to cartesian coordinates

In [12]:
lat, lon = Geometry.to_lat_lon(hcat(line1, sample1), height, slc1_pod[1], slc1_pod[2], slc1_meta)
pixel_position = Geometry.ellipsoid2xyz(deg2rad(lat), deg2rad(lon), height...)

(3.5714867991038603e6, 481840.1801384752, 5.24476468370043e6)

#### Compute the satellite position at the azimuth time corresponding to the line of interest

In [21]:
azimuth_time = slc1_meta["t_start"] + (line1[1] - 1) * 1 / slc1_meta["azimuth_frequency"]
state_vectors_poly, state_vectors_mean, state_vectors_std = Geometry.satellite_trajectory(slc1_pod[1], slc1_pod[2], slc1_meta["t_start"], slc1_meta["t_stop"]);
tmp = Geometry.polyval_state_vectors(state_vectors_poly, azimuth_time, state_vectors_mean, state_vectors_std)
sat1_position, sat1_velocity = tmp[1:3], tmp[4:6];

#### Compute line of sight 1

In [22]:
LOS1 = sat1_position .- pixel_position

3-element Array{Float64,1}:
 376165.2779512401 
 691432.9649579779 
 499507.38678614516

#### Compute line sample in slc2 geometry from the lat, lon point

In [15]:
line2, sample2 = to_line_sample(hcat(lat, lon), height, slc2_pod[1], slc2_pod[2], slc2_meta)

1×2 Array{Float64,2}:
 8530.25  12682.3

#### Compute the satellite position at the azimuth time corresponding to the line of interest

In [16]:
azimuth_time = slc2_meta["t_start"] + (line2[1] - 1) * 1 / slc2_meta["azimuth_frequency"]
state_vectors_poly, state_vectors_mean, state_vectors_std = Geometry.satellite_trajectory(slc2_pod[1], slc2_pod[2], slc2_meta["t_start"], slc2_meta["t_stop"]);
tmp = Geometry.polyval_state_vectors(state_vectors_poly, azimuth_time, state_vectors_mean, state_vectors_std)
sat2_position, sat2_velocity = tmp[1:3], tmp[4:6];

#### Compute line of sight 2

In [17]:
LOS2 = sat2_position .- pixel_position

3-element Array{Float64,1}:
 376166.50496910745
 691495.0362815189 
 499495.7727967473 

#### Calculate the perpenducular baseline.
The sign keeps sign across many baseline computations consistent wrt to a given master. The norm of the cross product is the area of the parallelogram spanned by the direct baseline btw slc1 and slc2 and the unit vector of slc1. Hence the area only depends on length of perpendicular baseline btw slc 1 and slc2.

In [23]:
s = sign(norm(LOS1) - norm(LOS2));

In [24]:
bperp = s * norm(cross(LOS1 - LOS2, LOS1 ./ norm(LOS1)))

-48.624954511700345

#### The days btw the acquisitions

In [20]:
days_between_acquisitions(extract_datetime(slc1_safe_path), extract_datetime(slc2_safe_path))

-12